In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

sys.path.append(os.path.abspath('../'))

del sys, os

In [3]:
import pandas as pd
import numpy as np

from torch import nn

from time import time

In [4]:
import matplotlib.pyplot as plt

In [5]:
# LaTeX rendering in graphs
from distutils.spawn import find_executable

if find_executable('latex'):
    plt.rc('text', usetex=True)
else:
    print('LaTeX not found...')

plt.rc('font', family='serif')

# High resolution graphs
%config InlineBackend.figure_format = 'retina'

# VAE

### Model Training

We ran the training of our model on GPU, using the specified hyperparameters and architecture. We obtain a validation ELBO of -95.90, which corresponds to the desired objective. 

### Evaluating log-likelihood with Importance Samping

In [7]:
'''
test=False
writer = SummaryWriter('logs/vae')

train_set = dataset.BinarizedMNIST(data_dir='../data/', split='train')
valid_set = dataset.BinarizedMNIST(data_dir='../data/', split='valid')

if test:
    train_set = Subset(train_set, indices=np.arange(200))
    valid_set = Subset(valid_set, indices=np.arange(200))

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=10)
valid_loader = DataLoader(valid_set, batch_size=128, shuffle=True, num_workers=10)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = vae.VariationalAutoEncoder()
model = model.to(device)

optimiser = Adam(model.parameters(), lr=3e-4)
'''

"\ntest=False\nwriter = SummaryWriter('logs/vae')\n\ntrain_set = dataset.BinarizedMNIST(data_dir='../data/', split='train')\nvalid_set = dataset.BinarizedMNIST(data_dir='../data/', split='valid')\n\nif test:\n    train_set = Subset(train_set, indices=np.arange(200))\n    valid_set = Subset(valid_set, indices=np.arange(200))\n\ntrain_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=10)\nvalid_loader = DataLoader(valid_set, batch_size=128, shuffle=True, num_workers=10)\n\ndevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\nmodel = vae.VariationalAutoEncoder()\nmodel = model.to(device)\n\noptimiser = Adam(model.parameters(), lr=3e-4)\n"

In [9]:
writer = SummaryWriter('logs/vae')
train_set = dataset.BinarizedMNIST(data_dir='../data/', split='train')
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=10)

In [10]:
import models.vae as vae
import learning.run_vae as run
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = vae.VariationalAutoEncoder(kappa = 0.5)
model = model.to(device)
optimiser = Adam(model.parameters())
#optimiser = Adam(model.parameters(), lr=.0003)

In [11]:
for _ in range(10):
    run.train(model, device, 1, train_loader, optimiser, writer)

/home/vincent/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


loss = 0.25153860449790955, div = 0.0025087567046284676, rec = 0.25028422474861145, norm = 0, time 20 batch = 1.0687265396118164
loss = 0.14837512373924255, div = 0.004232295788824558, rec = 0.14625898003578186, norm = 0, time 20 batch = 1.9888415336608887
loss = 0.11691482365131378, div = 0.008122491650283337, rec = 0.11285357922315598, norm = 0, time 20 batch = 2.890936851501465
loss = 0.09470321983098984, div = 0.0033870553597807884, rec = 0.09300969541072845, norm = 0, time 20 batch = 3.7926371097564697
loss = 0.08629295974969864, div = 0.003841399447992444, rec = 0.08437225967645645, norm = 0, time 20 batch = 4.696412563323975
loss = 0.09139902889728546, div = 0.006884623784571886, rec = 0.08795671910047531, norm = 0, time 20 batch = 5.600386381149292
loss = 0.09070780873298645, div = 0.005752307828515768, rec = 0.08783165365457535, norm = 0, time 20 batch = 6.504040002822876
loss = 0.0857718363404274, div = 0.0062501076608896255, rec = 0.08264677971601486, norm = 0, time 20 batch

KeyboardInterrupt: 

In [12]:
model = model.cpu()
for i,batch in enumerate(train_loader):
    out, mean, logv = model(batch)
    break

samples = 200
batch.size()

torch.Size([64, 1, 28, 28])

In [13]:
sigma = torch.exp(.5 * logv) + .1e-8
m = torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([1.0]))
sigma.size()

torch.Size([64, 100])

In [17]:
 # Sample hiddens and get proba for importance normalisation
samples_value_normal = torch.randn((samples, batch.size()[0], mean.size()[1]))
proba_q = m.cdf(samples_value_normal)
aggregated_q = proba_q.log().sum(dim=2)
aggregated_q.size()

torch.Size([200, 64])

In [18]:
mean[0] + samples_value_normal[0,0] * sigma[0]

tensor([-1.4808, -0.5995,  1.1857, -2.1320, -0.2487, -0.4530,  0.5929, -1.5606,
         1.0839, -0.4116, -0.0087, -0.9208, -1.7483, -2.1671, -0.1771,  0.2527,
         1.6754, -0.7516,  1.5345, -0.8301,  1.1191,  0.8103,  0.4717,  0.5474,
         0.4394,  1.4213, -0.7487, -0.6287, -0.4643,  0.8647, -1.1458, -0.2447,
         1.6241,  1.1901,  0.1866,  0.1872,  1.7081,  0.5379, -0.0144,  1.0677,
        -0.2636,  0.0063,  1.3551, -1.6580, -1.8167,  1.7688, -0.9652,  0.2420,
        -0.7793, -1.8211,  1.0559,  0.8359,  0.1747,  2.2991,  1.2314,  0.5664,
         0.6848, -0.0759, -0.5916,  0.4499,  0.5054, -1.8867, -0.5988, -1.7478,
        -2.3239, -0.5859, -1.2508, -0.6572,  2.0412,  2.5937,  0.9449, -0.6243,
        -0.6858, -1.6534,  0.7329, -0.9070,  0.1740, -0.5129, -0.6741, -0.0885,
        -0.7740,  0.3672,  1.3538, -0.2847,  0.2360, -0.7668, -0.5703, -0.1505,
         0.4553,  0.7291,  0.0620, -1.2990, -1.5556,  1.7671, -0.6017,  0.1702,
         1.3544,  1.2044,  1.2978,  1.52

In [19]:
 # Get prior proba of these hidden
value_z = mean + samples_value_normal * sigma
proba_prior_z = m.cdf(value_z)
aggregated_z = proba_prior_z.log().sum(dim=2)
print(aggregated_z.size())

torch.Size([200, 64])


In [20]:
batch.size()

torch.Size([64, 1, 28, 28])

In [34]:
 # Get actual conditional proba of data, one 'x' at a time. We decode the 200 samples and compute the ll of each.
cond_proba = list()
for i in range(value_z.size()[1]):
    with torch.no_grad():
        print(i)
        estimated_conditional_point = model.decode(value_z[:,i,:]).view(-1,784)
        #print(estimated_conditional_point.size())
        #print(estimated_conditional_point[0])
        x = batch[i].view(-1,784)
        proba_conditional = x * estimated_conditional_point + (1 - x) * (1 - estimated_conditional_point)
        #print(proba_conditional.size())
        aggregated_cond = proba_conditional.log().sum(dim=1)
        #print(aggregated_cond.size())
        cond_proba.append(aggregated_cond)


0


/home/vincent/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [42]:
proba_tensor = torch.stack(cond_proba).transpose(0,1)
proba_tensor.size()

torch.Size([200, 64])

In [51]:
proba_points = (aggregated_z + proba_tensor - aggregated_q).exp()
proba_points.mean(dim=0)
proba_points

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<ExpBackward>)